# Laptop Stand: face detection and gaze orientation
Alena, Siying, Kris

## Overview
tutorial: https://www.assemblyai.com/blog/mediapipe-for-dummies


In [ ]:
# !pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
import urllib.request
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
import PyQt5
from PIL import Image
from IPython.display import Video

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh

In [ ]:
# ref: https://unsplash.com/photos/smiling-woman-standing-while-holding-orange-folder-FcLyt7lW5wg
img = Image.open('student_photo.jpg')
display(img)

In [ ]:
file='student_photo.jpg'
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Create a face mesh object
with mp_face_mesh.FaceMesh(
        static_image_mode=True, # change to dynamic
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:

    image = cv2.imread(file)
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

face_found = bool(results.multi_face_landmarks)

In [ ]:
face_found

In [ ]:
if face_found:
    annotated_image = image.copy()
    
    mp_drawing.draw_landmarks(
        image=annotated_image,
        landmark_list=results.multi_face_landmarks[0],
        connections=mp_face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
    connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1))
        
    # Save image
    cv2.imwrite('face_tesselation_only.png', annotated_image)

# Open image
img = Image.open('face_tesselation_only.png')
display(img)

In [ ]:
# Specify the image filename
file = 'student_photo.jpg'

# Create a MediaPipe `Pose` object
with mp_pose.Pose(static_image_mode=True, 
		  model_complexity=2,
                  enable_segmentation=True) as pose:
        
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

annotated_image = image.copy()

mp_drawing.draw_landmarks(annotated_image, 
                          results.pose_landmarks, 
                          mp_pose.POSE_CONNECTIONS,
                          landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                          connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2))

filename = "pose_wireframe.png"
cv2.imwrite(filename, annotated_image)

display(Image.open(filename))

In [ ]:
UPPER_BODY_LANDMARKS = [
    mp_holistic.PoseLandmark.nose_3d,
    # mp_holistic.PoseLandmark.LEFT_EYE_INNER,
    mp_holistic.PoseLandmark.LEFT_EYE,
    # mp_holistic.PoseLandmark.LEFT_EYE_OUTER,
    # mp_holistic.PoseLandmark.RIGHT_EYE_INNER,
    mp_holistic.PoseLandmark.RIGHT_EYE,
    # mp_holistic.PoseLandmark.RIGHT_EYE_OUTER,
    mp_holistic.PoseLandmark.LEFT_EAR,
    mp_holistic.PoseLandmark.RIGHT_EAR,
    mp_holistic.PoseLandmark.LEFT_SHOULDER,
    mp_holistic.PoseLandmark.RIGHT_SHOULDER
]

In [ ]:
landmarks = results.pose_world_landmarks.landmark

upper_body_data = {}
for lm in UPPER_BODY_LANDMARKS:
    name = lm.name  # e.g., 'LEFT_SHOULDER'
    x, y, z = landmarks[lm.value].x, landmarks[lm.value].y, landmarks[lm.value].z
    upper_body_data[name] = (x, y, z)


In [ ]:
upper_body_data

In [ ]:
file = 'student_video.mp4'
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture(file)
	# Get the number of frames in the video
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Create a NumPy array to store the pose data as before
    # The shape is 3x33x144 - 3D XYZ data for 33 landmarks across 144 frames
    data = np.empty((3, len(UPPER_BODY_LANDMARKS), length))    
    
	# For each image in the video, extract the spatial pose data and save it in the appropriate spot in the `data` array 
    frame_num = 0
    while cap.isOpened():
        ret, image = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        landmarks = results.pose_world_landmarks.landmark
        for i, lm_id in enumerate(UPPER_BODY_LANDMARKS):
            data[:, i, frame_num] = (
                landmarks[i].x,
                landmarks[i].y,
                landmarks[i].z
            )  
        
        frame_num += 1
    
    # Close the video file
    cap.release()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig = plt.figure()
fig.set_size_inches(5, 5, True)
ax = fig.add_subplot(projection='3d')

# Setup the 3D axes
ax.set_xlim3d(-1, 1)
ax.set_ylim3d(-1, 1)
ax.set_zlim3d(-1, 1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Initial frame
x = data[0, :, 0]
y = data[1, :, 0]
z = data[2, :, 0]
points, = ax.plot(x, y, z, 'o-', color='blue')

# Define an update function for each frame
def update(frame):
    points.set_data(data[0, :, frame], data[1, :, frame])
    points.set_3d_properties(data[2, :, frame])
    return points,

# Create the animation
anim = FuncAnimation(fig, update, frames=data.shape[2],
                     interval=33, blit=True)

from matplotlib.animation import FFMpegWriter

# Set up the writer explicitly
writer = FFMpegWriter(fps=30, codec='libx264', extra_args=['-pix_fmt', 'yuv420p'])

# Save the animation
anim.save('walking_wireframe.mp4', writer=writer, dpi=300) # note : might bneed to ru n brew install ffmpeg


In [ ]:
def landmarks_to_control(landmarks, k=0.5, target=(0.0, 0.2, -0.3)):
    """
    Args:
    -   landmarks: 3d coord of body parts
    -   k: proportion of adjustment; k in (0, 1]
    -   target_pos: ideal location of 3d_nose
        Default centered nose position:
        x = 0.0
        y = 0.2             # slightly above body's COM
        z = -0.3            # slightly in front of the body's COM

    
    Returns:
    adjustment in 3D world coordinates
    """

    nose_3d = np.array(landmarks['NOSE'])
    eye_mid_3d = (np.array(landmarks['LEFT_EYE']) + np.array(landmarks['RIGHT_EYE'])) / 2.0
    left_sh_3d = np.array(landmarks['LEFT_SHOULDER'])
    right_sh_3d = np.array(landmarks['RIGHT_SHOULDER'])
    shoulder_mid_3d = (left_sh_3d + right_sh_3d) / 2.0
    
    # body vector (torso -> head)
    #     o|o               o|o
    #    _/_                _|_
    #    /          ->       |
    #   =                    =
    #  ||                   ||
    view_vector_3d = (nose_3d + eye_mid_3d) / 2.0 - shoulder_mid_3d
    view_vector_3d /= np.linalg.norm(view_vector_3d)
    
    # offsets rel to laptop position at origin, assume screen centers at (0,0,0)
    horizontal_error = nose_3d[0] # 3d coodinates
    vertical_error   = nose_3d[1]
    depth_error      = nose_3d[2]  # negative = farther

    pitch = np.arctan2(view_vector_3d[1], view_vector_3d[2])
    yaw   = np.arctan2(view_vector_3d[0], view_vector_3d[2])
    
    # control command ~ adjustments
    adjustments = {
        'dx': -k * horizontal_error,
        'dy': -k * vertical_error,
        'dz': -k * depth_error,        # optional if stand can move forward/back
        'tilt_pitch': -np.degrees(pitch),  # negative to align screen with view
        'tilt_yaw': -np.degrees(yaw)
    }
    
    return adjustments



In [ ]:
PoseLandmark = mp.solutions.pose.PoseLandmark
POSE_TO_I = {
	'NOSE': PoseLandmark.NOSE.value,
	'LEFT_SHOULDER': PoseLandmark.LEFT_SHOULDER.value,
	'RIGHT_SHOULDER': PoseLandmark.RIGHT_SHOULDER.value,
}

In [ ]:
# draw adj
def project_to_image(point_3d, scale, img_center_2d):
    """
    Simple orthographic projection to 2D image space
    scale: pixels per meter
    img_center_2d: center of the image
    """
    if not img_center_2d or not scale:
        print("no img center or scale given")
        return None
    x = int(img_center_2d[0] + point_3d[0] * scale)
    y = int(img_center_2d[1] - point_3d[1] * scale)  # y-axis inverted for image
    return (x, y)

def get_frame_params(results, h, w, left_sh_3d, right_sh_3d):
    """
    Args:
    - results: MediaPipe model

    Estimates body's COM as a middle between shoulders. It's techincally mid of hips, but we don't capture them in the photo.
    Estimates scale (#pixels per meter): ratio of distance between shoulders in pixels vs in 3d coordinates
    """
    left_shoulder_2d = results.pose_landmarks.landmark[POSE_TO_I['LEFT_SHOULDER']] # in img frame
    right_shoulder_2d = results.pose_landmarks.landmark[POSE_TO_I['RIGHT_SHOULDER']]
    shoulder_mid_2d = ((left_shoulder_2d.x + right_shoulder_2d.x) / 2, (left_shoulder_2d.y + right_shoulder_2d.y) / 2)
    img_center_2d = shoulder_mid_2d

    left_sh_px = np.array([left_shoulder_2d.x * w, left_shoulder_2d.y * h])
    right_sh_px = np.array([right_shoulder_2d.x * w, right_shoulder_2d.y * h])
    pixel_distance = np.linalg.norm(left_sh_px - right_sh_px) # image frame
    real_distance = np.linalg.norm(left_sh_3d - right_sh_3d) # 3d world frame
    scale = pixel_distance / real_distance # pixels per meter

    return img_center_2d, scale

# example:
file = 'student_photo.jpg'

# Create a MediaPipe `Pose` object
with mp_pose.Pose(static_image_mode=True, model_complexity=2,
                  enable_segmentation=True) as pose:
    upper_body_data_3d = {} # lm name -> 3d-coord tuple

    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    world_landmarks = results.pose_world_landmarks.landmark
   
    for lm in UPPER_BODY_LANDMARKS:
        i = lm.value # gives index in landmarks
        x, y, z = world_landmarks[i].x, world_landmarks[i].y, world_landmarks[i].z
        upper_body_data_3d[lm.name] = (x, y, z) # in 3d coord

print("Upper body data in 3d: ", upper_body_data_3d)
adjustments_3d = landmarks_to_control(upper_body_data_3d, k=0.7) # 3d coord
print("Adjustment: ", adjustments_3d)

nose_3d = np.array(upper_body_data_3d['NOSE'])
left_sh_3d = np.array(upper_body_data_3d['LEFT_SHOULDER'])
right_sh_3d = np.array(upper_body_data_3d['RIGHT_SHOULDER'])
shoulder_mid_3d = (left_sh_3d + right_sh_3d) / 2.0

h, w, _ = image.shape
img_center_2d, scale = get_frame_params(results, h, w, left_sh_3d, right_sh_3d)

view_vector_3d = nose_3d - shoulder_mid_3d # in 3d
print("View vector: ", view_vector_3d)
nose_2d = project_to_image(nose_3d, scale, img_center_2d) # 2d projection; we can also get in directly as above
shoulder_2d = project_to_image(shoulder_mid_3d, scale, img_center_2d)

# draw pose and adjustment vector
annotated_image = image.copy()
cv2.circle(annotated_image, center=nose_2d, radius=50, color=(0, 0, 255), thickness=2)
cv2.circle(annotated_image, center=shoulder_2d, radius=50, color=(0, 255, 0), thickness=2)

# adjustment vector
cv2.arrowedLine(annotated_image, shoulder_2d, nose_2d, (255, 0, 0), 2, tipLength=0.2)

filename = "pose_student.png"
cv2.imwrite(filename, annotated_image)
cv2.destroyAllWindows()
display(Image.open(filename))

In [ ]:
import time

def activate_actuator_speeds(control_signal_x, control_signal_y, control_signal_pitch, control_signal_yaw):
	return # to do

class PID:
	def __init__(self, Kp, Ki, Kd, target=0.0):
		self.Kp = Kp
		self.Ki = Ki
		self.Kd = Kd
		self.target = target
		self.integral = 0.0
		self.prev_error = 0.0

	def update(self, observed, dt):
		error = self.target - observed
		self.integral += error * dt
		if dt > 0:
				derivative = (error - self.prev_error) / dt
		else:
				derivative = 0
		self.prev_error = error

		control_signal = (self.Kp * error) + (self.Ki * self.integral) + (self.Kd * derivative)
		return control_signal

pid_x = PID(1.0, 0.01, 0.1)
pid_y = PID(1.0, 0.01, 0.1)
pid_pitch = PID(2.0, 0.02, 0.2)
pid_yaw = PID(2.0, 0.02, 0.2)

# control cycle loop
dt = 0.01  # 10ms
while True:
	adjustments = landmarks_to_control(landmarks, k=1, target) # 3D world coordina; k=1 as step size handled in PID
	x_observed = adjustments.dx
	y_observed = adjustments.dy
	# z_observed = adjustments.dz ??
	pitch_observed = adjustments.tilt_pitch
	yaw_observed = adjustments.tilf_yaw
	
	control_signal_x = pid_x.update(x_observed, dt)
	control_signal_y = pid_y.update(y_observed, dt)
	control_signal_pitch = pid_pitch.update(pitch_observed, dt)
	control_signal_yaw = pid_yaw.update(yaw_observed, dt)
	
	activate_actuator_speeds(control_signal_x, control_signal_y, control_signal_pitch, control_signal_yaw) # TODO
	time.sleep(dt)
